# 03 - Federated Learning

## Defines

Define the available types of federated learning.

 - 'STRATIFIED': Stratified sampling of the data. The data is split into a number of shards, and each shard is assigned to a client. The data is split in a stratified manner, meaning that the distribution of the labels is approximately the same in each shard.
 - 'MISSING_1_ATTACK' - Each client is assigned a shard of data, each shard is missing one of the attack labels. Other clients in the network are exposed to the attack label, but the specific client is not. This demonstrates the ability of federated learning to protect against unknown attacks.
 - '1_ATTACK_ONLY' - Each client is assigned a shard of data, each shard contains only one of the attack labels.
 - 'HALF_BENIGN_ONLY' - Half of the clients are exposed to Benign data only, the other half are exposed to all data.


In [63]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['STRATIFIED','MISSING_1_ATTACK', '1_ATTACK_ONLY', 'HALF_BENIGN_ONLY' ]
METHOD = 'MISSING_1_ATTACK'
NUM_OF_STRATIFIED_CLIENTS = 10  # only applies to stratified method
NUM_OF_ROUNDS = 10              # Number of FL rounds


The above test method in conjunction with the below classification selection will determine the number of clients.

EG: 
`STRATIFIED` with:
 - `ALL TYPES` - Results in `NUM_OF_STRATIFIED_CLIENTS` clients. Each client will have a stratified sample of the data.

`MISSING_1_ATTACK` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 32 attack labels.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 6 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`1_ATTACK_ONLY` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 1 attack label.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 1 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`HALF_BENIGN_ONLY` with:
 - `individual_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 33 malicious attack labels.
 - `group_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 7 malicious attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and Malicious attack labels.

In [64]:
individual_classifier = False
group_classifier = True
binary_classifier = False


Include the defines for the dataframe columns and the attack labels and their mappings

In [65]:
from enum import Enum
from includes import *

##  Imports

In [66]:
%%capture
%pip install flwr[simulation] torch torchvision matplotlib sklearn openml

In [67]:
import os
import pandas as pd
import numpy as np
import flwr as fl
from tqdm import tqdm
import warnings
#warnings.filterwarnings('ignore')

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split


In [68]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.4.0
numpy 1.23.5
torch 2.0.1+cpu
Training on cpu


Prepare an output directory where we can store the results of the federated learning

In [ ]:
# Create an "Output" directory if it doesnt exist already
if not os.path.exists("Output"):
    os.makedirs("Output")

sub_directory_name = f"Output/{METHOD}_Classifier-{class_size}_Clients-{NUM_OF_CLIENTS}_Rounds-{NUM_OF_ROUNDS}"

# Create an "Output/{METHOD}-{NUM_OF_CLIENTS}-{NUM_OF_ROUNDS}" directory if it doesnt exist already
if not os.path.exists(f"Output/{sub_directory_name}"):
    os.makedirs(f"Output/{sub_directory_name}")

# Ensure the directory is empty
for file in os.listdir(f"Output/{sub_directory_name}"):
    file_path = os.path.join(f"Output/{sub_directory_name}", file)
    if os.path.isfile(file_path):
        os.unlink(file_path)

## Load the Dataset

In [69]:
DATASET_DIRECTORY = '../datasets/CICIoT2023/'

## Training data

Either read the training pickle file if it exists, or process the dataset from scratch.

In [70]:
# Check to see if the file 'training_data.pkl' exists in the directory. If it does, load it. If not, print an error.
if os.path.isfile('training_data.pkl'):
    print("File exists, loading data...")
    train_df = pd.read_pickle('training_data.pkl')
    print("Training data loaded from pickle file.")

else:
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    # Print the number of files in each set
    print('Training sets: {}'.format(len(training_sets)))
    print('Test sets: {}'.format(len(test_sets)))

    ######################
    # HACK TEMP CODE
    ######################
    # Set training_sets to the last entry of training_sets
    training_sets = training_sets[-5:]
    print(f"HACK TO REPLICATE ORIGINAL AUTHORS CODE WITH ONE FILE TRAIN - {training_sets}")
    ######################
    # HACK END TEMP CODE
    ######################

    # Concatenate all training sets into one dataframe
    dfs = []
    print("Reading training data...")
    for train_set in tqdm(training_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + train_set)
        dfs.append(df_new)
    train_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    train_df['label'] = train_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print("Writing training data to pickle file...")
    train_df.to_pickle('training_data.pkl')

print("Training data size: {}".format(train_df.shape))


File exists, loading data...
Training data loaded from pickle file.
Training data size: (1425287, 47)


In [71]:
train_df

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000000,182.00,17.00,64.00,22.362751,22.362751,0.0,0.0,0.0,0.0,...,0.000000,182.00,8.300743e+07,9.5,19.078784,0.000000,0.000000,0.00,141.55,13
1,2.437778,129.60,6.00,64.00,0.978382,0.978382,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.336252e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,7
2,0.000000,54.00,6.00,64.00,0.000000,0.000000,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334496e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,1
3,0.453670,39173.00,17.00,64.00,4967.422026,4967.422026,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.310643e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,4
4,0.000000,54.00,6.00,64.00,166.930829,166.930829,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.331469e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425282,0.000000,54.00,6.00,64.00,19.582485,19.582485,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.331443e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,2
1425283,0.037146,78.22,36.21,63.18,24.542045,24.542045,0.0,0.0,0.0,0.0,...,110.233513,453.78,8.358187e+07,9.5,30.338676,154.660856,23401.960226,0.53,141.55,18
1425284,3.293075,1025996.92,17.00,64.00,572.160392,572.160392,0.0,0.0,0.0,0.0,...,0.000000,554.00,8.378910e+07,9.5,33.286634,0.000000,0.000000,0.00,141.55,19
1425285,0.047343,35223.00,17.00,64.00,15083.107398,15083.107398,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.309852e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,4


---
## Test Data
Concat the test data into a single dataframe

In [72]:
# Check to see if the file 'test_data.pkl' exists in the directory. If it does, load it. If not, print an error.
testing_data_pickle_file = 'testing_data.pkl'

if os.path.isfile(testing_data_pickle_file):
    print(f"File {testing_data_pickle_file} exists, loading data...")
    test_df = pd.read_pickle(testing_data_pickle_file)
    print("Test data loaded from pickle file.")

else:
    print(f"File {testing_data_pickle_file} does not exist, constructing data...")

    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    ############################################
    ############################################
    # HACK - Make things quicker for now
    ############################################
    ############################################

    test_sets = df_sets[int(len(df_sets)*.95):]
    
    # Set training_sets to the last entry of training_sets
    test_sets = test_sets[-2:]
    
    ############################################
    ############################################
    # END HACK 
    ############################################
    ############################################

    # Print the number of files in each set
    print('Test sets: {}'.format(len(test_sets)))
    
    # Concatenate all testing sets into one dataframe
    dfs = []
    print("Reading test data...")
    for test_set in tqdm(test_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + test_set)
        dfs.append(df_new)
    test_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    test_df['label'] = test_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print(f"Writing test data to pickle file {testing_data_pickle_file}...")
    test_df.to_pickle(testing_data_pickle_file)

print("Testing data size: {}".format(test_df.shape))

File testing_data.pkl exists, loading data...
Test data loaded from pickle file.
Testing data size: (462433, 47)


---
# Scale the test and train data

### Scale the training data input features

In [73]:
scaler = StandardScaler()
train_df[X_columns] = scaler.fit_transform(train_df[X_columns])

### Scale the testing data input features

In [74]:
test_df[X_columns] = scaler.fit_transform(test_df[X_columns])

---
# Define the classification problem - (2 classes, 8 classes or 34 classes)
Change the following cell to select the classification type

If the METHOD == STRATIFIED, then we can use any classifier
If the METHOD == ATTACK_GROUP then we must use Group Classifier.

In [75]:

class_size_map = {2: "Binary", 8: "Group", 34: "Individual"}

if group_classifier:
    print("Group 8 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_7_classes values
    test_df['label'] = test_df['label'].map(dict_8_classes)
    train_df['label'] = train_df['label'].map(dict_8_classes)
    class_size = "8"
        
elif binary_classifier:
    print("Binary 2 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_2_classes values
    test_df['label'] = test_df['label'].map(dict_2_classes)
    train_df['label'] = train_df['label'].map(dict_2_classes)
    class_size = "2"

else:
    print ("Individual 34 Class classifier... - No adjustments to labels in test and train dataframes")
    class_size = "34"

Group 8 Class Classifier... - Adjusting labels in test and train dataframes


---
# Split the Training Data into partitions for the Federated Learning clients depending on the test required
As a reminder:

`STRATIFIED` with:
 - `ALL TYPES` - Results in `NUM_OF_STRATIFIED_CLIENTS` clients. Each client will have a stratified sample of the data.

`MISSING_1_ATTACK` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 32 attack labels.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 6 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`1_ATTACK_ONLY` with:
 - `individual_classifier` - Results in 33 clients. Each client will have benign traffic and 1 attack label.
 - `group_classifier` - Results in 7 clients. Each client will have benign traffic and 1 attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and malicious attack labels.

`HALF_BENIGN_ONLY` with:
 - `individual_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 33 malicious attack labels.
 - `group_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and 7 malicious attack groups.
 - `binary_classifier` - Results in 10 clients. Five clients will have benign traffic only and the other will have Benign and Malicious attack labels.

In [76]:
from sklearn.model_selection import StratifiedKFold

# Define fl_X_train and fl_y_train
fl_X_train = []
fl_y_train = []

if METHOD == 'STRATIFIED':
    print(f"{Colours.YELLOW.value}STRATIFIED METHOD{Colours.NORMAL.value} with {class_size} class classifier")
    # We are going to split the training data into 'NUM_OF_STRATIFIED_CLIENTS' smaller groups using StratifiedKFold
    skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(train_df[X_columns], train_df[y_column]):
        fl_X_train.append(train_df[X_columns].iloc[test_index])
        fl_y_train.append(train_df[y_column].iloc[test_index])

elif METHOD == 'MISSING_1_ATTACK':
    print(f"{Colours.YELLOW.value}MISSING_1_ATTACK METHOD{Colours.NORMAL.value} with {class_size} class classifier")

    if individual_classifier or group_classifier:
        # Set the number of splits required to the number of classes - 1
        num_splits = int(class_size) - 1
    else:
        # For binary classifier, set the number of splits to 10
        num_splits = 10

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

    # When creating the clients, we will remove one attack class from the training data
    # For the binary classifier, evey other client will have the benign class removed
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        if binary_classifier:
            print(f"i: {i} = i % 2 = {i % 2}")
            if i % 2 == 0:
                print("Benign only")
                # Create a new dataframe for the client data with only benign traffic
                client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != 1]], ignore_index=True)
                fl_X_train.append(client_df[X_columns])
                fl_y_train.append(client_df[y_column])
            else:
                print("Both")
                # Create a new dataframe for the client data
                fl_X_train.append(train_df[X_columns].iloc[test_index])
                fl_y_train.append(train_df[y_column].iloc[test_index])
        else:
            # Create a new dataframe for the client data
            client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i+1]], ignore_index=True)
            fl_X_train.append(client_df[X_columns])
            fl_y_train.append(client_df[y_column])

elif METHOD == 'ATTACK_GROUP':
    print(f"{Colours.YELLOW.value}ATTACK_GROUP METHOD{Colours.NORMAL.value}")
    # With this method we split the data so that each client data sees all attacks except one. 
    # All clients will see attack traffic BenignTraffic - 0.
    # EG:
    # client 0 will see attacks 2-7
    # client 1 will see attacks 1, 3-7
    # client 2 will see attacks 1-2, 4-7
    
    # There are 7 attack groups + 1 benign class, so we will create 7 clients
    skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
    for i, (train_index, test_index) in enumerate(skf.split(train_df[X_columns], train_df[y_column])):
        # Create a new dataframe for the client data
        client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i+1]], ignore_index=True)
        fl_X_train.append(client_df[X_columns])
        fl_y_train.append(client_df[y_column])
        
    pass  



MISSING_1_ATTACK METHOD with 8 class classifier


C:\Users\jd7-white\AppData\Local\Temp\ipykernel_21296\1879862568.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i+1]], ignore_index=True)
C:\Users\jd7-white\AppData\Local\Temp\ipykernel_21296\1879862568.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i+1]], ignore_index=True)
C:\Users\jd7-white\AppData\Local\Temp\ipykernel_21296\1879862568.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i+1]], ignore_index=True)
C:\Users\jd7-white\AppData\Local\Temp\ipykernel_21296\1879862568.py:45: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  client_df = pd.concat([train_df.iloc[test_index][train_df[y_column] != i+1]], ignore_index=True)
C:\Users\jd7

In [86]:
for i in range(len(fl_X_train)):
    # Show the unique values in the y column
    (f"Client ID: {i}")
    print(f"fl_X_train[{i}].shape: {fl_X_train[i].shape}")  
    print(f"fl_y_train[{i}].value_counts():\n{fl_y_train[i].value_counts()}")
    print(f"fl_y_train[{i}].unique(): {fl_y_train[i].unique()}\n")

# Write this same info to the output directory/Class Split Info.txt
with open(f"Output/{sub_directory_name}/Class Split Info.txt", "w") as f:
    for i in range(len(fl_X_train)):
        f.write(f"Client ID: {i}\n")
        f.write(f"fl_X_train.shape: {fl_X_train[0].shape}\n")
        f.write(f"fl_y_train.value_counts():\n{fl_y_train[0].value_counts()}\n")
        f.write(f"fl_y_train.unique(): {fl_y_train[0].unique()}\n\n")


fl_X_train[0].shape: (55374, 46)
fl_y_train[0].value_counts():
7    35317
2    11442
0     4783
4     2090
3     1570
5      114
6       58
Name: label, dtype: int64
fl_y_train[0].unique(): [7 2 0 4 3 5 6]

fl_X_train[1].shape: (192171, 46)
fl_y_train[1].value_counts():
1    148239
7     35317
0      4783
4      2091
3      1569
5       114
6        58
Name: label, dtype: int64
fl_y_train[1].unique(): [1 7 4 3 0 5 6]

fl_X_train[2].shape: (202043, 46)
fl_y_train[2].value_counts():
1    148239
7     35317
2     11441
0      4783
4      2091
5       114
6        58
Name: label, dtype: int64
fl_y_train[2].unique(): [1 7 0 2 4 5 6]

fl_X_train[3].shape: (201521, 46)
fl_y_train[3].value_counts():
1    148239
7     35317
2     11441
0      4783
3      1570
5       113
6        58
Name: label, dtype: int64
fl_y_train[3].unique(): [7 1 3 0 2 5 6]

fl_X_train[4].shape: (203499, 46)
fl_y_train[4].value_counts():
1    148240
7     35316
2     11441
0      4783
4      2091
3      1570
6        58


### Convert the training dataset

In [78]:
# Convert the testing daya to X_test and y_test ndarrays
X_test = test_df[X_columns].to_numpy()
y_test = test_df[y_column].to_numpy()

---
### Data check

In [79]:
NUM_OF_CLIENTS = len(fl_X_train)
print("NUM_CLIENTS:", NUM_OF_CLIENTS)

print("NUM_ROUNDS:", NUM_OF_ROUNDS)
print()
print("Original train_df size: {}".format(train_df.shape))

print("Checking training data split groups")
for i in range(len(fl_X_train)):
    print(i, ":", "X Shape", fl_X_train[i].shape, "Y Shape", fl_y_train[i].shape)


# Print the sizes of X_test and y_test
print("\nChecking testing data")
print("X_test size: {}".format(X_test.shape))
print("y_test size: {}".format(y_test.shape))

print("\nDeploy Simulation")

NUM_CLIENTS: 7
NUM_ROUNDS: 10

Original train_df size: (1425287, 47)
Checking training data split groups
0 : X Shape (55374, 46) Y Shape (55374,)
1 : X Shape (192171, 46) Y Shape (192171,)
2 : X Shape (202043, 46) Y Shape (202043,)
3 : X Shape (201521, 46) Y Shape (201521,)
4 : X Shape (203499, 46) Y Shape (203499,)
5 : X Shape (203554, 46) Y Shape (203554,)
6 : X Shape (168296, 46) Y Shape (168296,)

Checking testing data
X_test size: (462433, 46)
y_test size: (462433,)

Deploy Simulation


----
# Federated Learning
## Import the libraries and print the versions

In [80]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

label = train_df[y_column]

Define the Client and Server code

In [82]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

print('scikit-learn {}.'.format(sklearn.__version__))
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

import datetime

client_evaluations = []

class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=5, batch_size=32)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels, batch_size=32)
        print(f"{Colours.YELLOW.value}Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}{Colours.NORMAL.value}")

        # Write the same message to the "Output/{cid}_Evaluation.txt" file
        with open(f"Output/{sub_directory_name}/{self.cid}_Evaluation.txt", "a") as f:
            f.write(f"{datetime.datetime.now()} - Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}\n")

            # Close the file
            f.close()

        return loss, len(self.train_data), {"accuracy": accuracy}
    
    def predict(self, incoming):
        prediction = np.argmax( self.model.predict(incoming) ,axis=1)
        return prediction

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(50, activation='relu'),  
      Dense(25, activation='relu'),  
      Dense(len(label.unique()), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

   
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
eval_count = 0

def get_evaluate_fn(server_model):
    global eval_count
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    
    
    def evaluate(server_round, parameters, config):
        global eval_count
        
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print (f"Server Evaluating... Evaluation Count:{eval_count}")
        loss, accuracy = server_model.evaluate(X_test, y_test)
        
        y_pred = server_model.predict(X_test)
        print ("Prediction: ", y_pred, y_pred.shape)
        #cmatrix = confusion_matrix(y_test, np.rint(y_pred))
        #print ("confusion_matrix:", cmatrix, cmatrix.shape)
                        
        print(f"{Colours.YELLOW.value}Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}{Colours.NORMAL.value}")

        # Write the same message to the "Output/Server_Evaluation.txt" file
        with open(f"Output/{sub_directory_name}/Server_Evaluation.txt", "a") as f:
            f.write(f"{datetime.datetime.now()} - {server_round} : Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}\n")

            # Close the file
            f.close()
        
        np.save("y_pred-" + str(eval_count) + ".npy", y_pred)
        #np.save("cmatrix-" + str(eval_count) + ".npy", cmatrix)
        eval_count = eval_count + 1
        
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
    #Flatten(input_shape=(79,1)),
    Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
    Dense(50, activation='relu'),  
    Dense(25, activation='relu'),  
    Dense(len(label.unique()), activation='softmax')
])


server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

scikit-learn 1.2.2.
flwr 1.4.0
numpy 1.23.5
tf 2.12.0


In [83]:
%%time
print (f"{Colours.YELLOW.value}\nDeploy simulation... Method = {METHOD} - {class_size_map[len(label.unique())]} ({class_size}) Classifier")
print (f"Number of Clients = {NUM_OF_CLIENTS}{Colours.NORMAL.value}\n")

# Output the same information to the Output/Run_details.txt file
with open(f"Output/{sub_directory_name}/Run_details.txt", "a") as f:
    f.write(f"{datetime.datetime.now()} - Deploy simulation... Method = {METHOD} - {class_size_map[len(label.unique())]} ({class_size}) Classifier\n")
    f.write(f"{datetime.datetime.now()} - Number of Clients = {NUM_OF_CLIENTS}\n")

    # Write Original train_df size
    f.write(f"{datetime.datetime.now()} - Original train_df size: {train_df.shape}\n")

    # Write the training data split groups
    for i in range(len(fl_X_train)):
        f.write(f"{datetime.datetime.now()} - {i}: X Shape {fl_X_train[i].shape}, Y Shape {fl_y_train[i].shape}\n")

    # Write the testing data
    f.write(f"{datetime.datetime.now()} - X_test size: {X_test.shape}\n")
    f.write(f"{datetime.datetime.now()} - y_test size: {y_test.shape}\n")
    
# close the file
f.close()

start_time = datetime.datetime.now()

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

end_time = datetime.datetime.now()
print("Total time taken: ", end_time - start_time)

print (f"{Colours.YELLOW.value} SIMULATION COMPLETE. Method = {METHOD} - {class_size_map[len(label.unique())]} ({class_size}) Classifier")
print (f"Number of Clients = {NUM_OF_CLIENTS}{Colours.NORMAL.value}\n")

# Output the same information to the Output/Run_details.txt file
with open(f"Output/{sub_directory_name}/Run_details.txt", "a") as f:
    f.write(f"{datetime.datetime.now()} - SIMULATION COMPLETE. Method = {METHOD} - {class_size_map[len(label.unique())]} ({class_size}) Classifier\n")
    f.write(f"{datetime.datetime.now()} - Total time taken: {end_time - start_time}\n")

INFO flwr 2023-07-09 17:35:12,311 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)



Deploy simulation... Method = MISSING_1_ATTACK - Group (8) Classifier
Number of Clients = 7

(launch_and_evaluate pid=3384) 3529/6314 [===============>..............] - ETA: 2s - loss: 1.3921 - accuracy: 0.8920 [repeated 6x across cluster]
(launch_and_evaluate pid=3384) 5010/6314 [======================>.......] - ETA: 1s - loss: 1.3853 - accuracy: 0.8921 [repeated 8x across cluster]
(launch_and_evaluate pid=3384) 5238/6314 [=======================>......] - ETA: 1s - loss: 1.3804 - accuracy: 0.8924 [repeated 7x across cluster]
(launch_and_evaluate pid=3384) 5429/6314 [========================>.....] - ETA: 0s - loss: 1.3843 - accuracy: 0.8921 [repeated 6x across cluster]
(launch_and_evaluate pid=3384) 5668/6314 [=========================>....] - ETA: 0s - loss: 1.3858 - accuracy: 0.8921 [repeated 7x across cluster]
(launch_and_evaluate pid=3384) 6314/6314 [==============================] - 6s 953us/step - loss: 1.3783 - accuracy: 0.8924 [repeated 2x across cluster]
(launch_and_evalua

2023-07-09 17:35:20,551	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-07-09 17:35:23,731 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 12.0, 'object_store_memory': 6771377356.0, 'node:127.0.0.1': 1.0, 'memory': 13542754715.0, 'GPU': 1.0}
INFO flwr 2023-07-09 17:35:23,732 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-09 17:35:23,733 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-07-09 17:35:30,575 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-07-09 17:35:30,575 | server.py:88 | Evaluating initial parameters


(launch_and_get_parameters pid=16536) Client ID: 5
Server Evaluating... Evaluation Count:0
14452/14452 [==============================] - 12s 845us/step


INFO flwr 2023-07-09 17:35:59,964 | server.py:91 | initial parameters (loss, other metrics): 2.5435526371002197, {'accuracy': 0.018037207424640656}
INFO flwr 2023-07-09 17:35:59,965 | server.py:101 | FL starting
DEBUG flwr 2023-07-09 17:35:59,967 | server.py:218 | fit_round 1: strategy sampled 7 clients (out of 7)


...

(launch_and_fit pid=16536) 5063/6362 [======================>.......] - ETA: 2s - loss: 0.0262 - accuracy: 0.9903 [repeated 34x across cluster]
(launch_and_fit pid=16536) 5277/6362 [=======================>......] - ETA: 2s - loss: 0.0265 - accuracy: 0.9901 [repeated 31x across cluster]
Server Evaluating... Evaluation Count:10
14452/14452 [==============================] - 12s 838us/step


INFO flwr 2023-07-09 17:54:50,673 | server.py:119 | fit progress: (10, 0.10946495085954666, {'accuracy': 0.9801225066184998}, 1130.7025980999751)
DEBUG flwr 2023-07-09 17:54:50,673 | server.py:168 | evaluate_round 10: strategy sampled 3 clients (out of 7)


Prediction:  [[2.8492358e-22 3.5085779e-04 7.7659303e-25 ... 1.3011635e-32
  6.4880605e-32 9.9964917e-01]
 [9.6239143e-20 9.9999809e-01 6.0948994e-13 ... 2.4500635e-27
  9.5894864e-31 1.9258762e-06]
 [1.6246824e-23 1.2571318e-01 2.9467840e-15 ... 4.0827115e-28
  3.4539165e-30 8.7428683e-01]
 ...
 [1.2271114e-24 7.3839515e-04 1.2004004e-18 ... 4.3983512e-29
  2.0735937e-31 9.9926156e-01]
 [3.4230534e-20 9.9999809e-01 3.9775797e-13 ... 7.5236988e-28
  3.4864356e-31 1.9599017e-06]
 [2.1616784e-31 1.0000000e+00 1.8091898e-32 ... 0.0000000e+00
  0.0000000e+00 2.4754354e-18]] (462433, 8)
Server evaluation complete - Accuracy: 0.9801, Loss: 0.1095
(launch_and_evaluate pid=16536) Client ID: 3
(launch_and_fit pid=16536) 4436/6362 [===================>..........] - ETA: 3s - loss: 0.0258 - accuracy: 0.9903 [repeated 29x across cluster]
(launch_and_fit pid=16536) 5477/6362 [========================>.....] - ETA: 1s - loss: 0.0266 - accuracy: 0.9901 [repeated 28x across cluster]
(launch_and_fit pi

DEBUG flwr 2023-07-09 17:54:58,679 | server.py:182 | evaluate_round 10 received 3 results and 0 failures
INFO flwr 2023-07-09 17:54:58,680 | server.py:147 | FL finished in 1138.7103187999746
INFO flwr 2023-07-09 17:54:58,681 | app.py:218 | app_fit: losses_distributed [(1, 0.7800800930338774), (2, 0.2286717738024931), (3, 0.8181537301830932), (4, 0.9541253002436343), (5, 0.8299552300467516), (6, 0.12508682987976197), (7, 0.25138312311840116), (8, 0.0829279356531834), (9, 0.09442021211437841), (10, 0.08563173001301279)]
INFO flwr 2023-07-09 17:54:58,682 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-07-09 17:54:58,682 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-07-09 17:54:58,683 | app.py:221 | app_fit: losses_centralized [(0, 2.5435526371002197), (1, 1.0462819337844849), (2, 0.3076600134372711), (3, 0.7712777256965637), (4, 0.7304874658584595), (5, 0.7058594226837158), (6, 0.12240297347307205), (7, 0.6991829872131348), (8, 0.13878796994686127), (9, 0

(launch_and_evaluate pid=16536) 2285/6298 [=========>....................] - ETA: 4s - loss: 0.0338 - accuracy: 0.9913 [repeated 8x across cluster]
(launch_and_evaluate pid=16536) 2519/6298 [==========>...................] - ETA: 4s - loss: 0.0331 - accuracy: 0.9915 [repeated 9x across cluster]
Total time taken:  0:19:46.378729
 SIMULATION COMPLETE. Method = MISSING_1_ATTACK - Group (8) Classifier
Number of Clients = 7

CPU times: total: 8min 55s
Wall time: 19min 46s
